In [ ]:
import gym
from gym import spaces
import numpy as np

class CustomEnv(gym.Env):
    def __init__(self, stock_data,windows=10,captial=300000):
        super(CustomEnv, self).__init__()

        # 加载股票数据
        self.initial_captial = captial
        self.stock_data = stock_data
        self.current_step = windows
        self.windows = windows
        
        # 定义观察空间和动作空间
        t = len(stock_data)
        n = len(stock_data[0])
        l = len(stock_data[0][0])
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(windows, n, l), dtype=np.float32)
        self.action_space = spaces.Box(low=0, high=1, shape=(n,), dtype=np.float32)

    def step(self, action):
        # 执行动作，返回新的状态、奖励和是否结束
        print('current_step:%s' % self.current_step)
        new_state = self.stock_data[self.current_step+1-self.windows:self.current_step+1]
        price = self.stock_data[self.current_step]
        
        reward_ = self.calculate_reward(price,action)
        if reward_ > 0:
            reward = reward_/self.captial
        else:
            reward = -1
        self.captial += reward_
        print('current_step:%s,earns:%f,captial:%f' % (self.current_step,reward_,self.captial))
        done = self.current_step+2 >= len(self.stock_data)
        self.current_step += 1
        return new_state, reward, done, {}

    def reset(self):
        # 重置环境，返回初始状态
        # 这部分需要根据你的具体需求来实现
        initial_state = self.stock_data[0:self.windows]
        
        self.captial = self.initial_captial
        self.current_step = self.windows
        return initial_state 

    def render(self, mode='human'):
        # 渲染环境（可选）
        pass

    def calculate_reward(self,price,action):
        # 计算回报
        open_price = price[:,0]
        pct_chg = price[:,6]
        
        action = action/np.sum(action)
        
        amounts = np.floor(action*self.captial/open_price/100)*100
        earns = 0
        
        valid_amounts = np.nan_to_num(amounts)  # 将NaN替换为0
        valid_open_price = np.nan_to_num(open_price)  # 将NaN替换为0
        consume = np.sum(valid_amounts * valid_open_price)
        print('consume:%f' % consume)
        
        for idx2,amount in enumerate(amounts):
            
            if amount>=100 and amount != np.inf:
                
                earn = amount*pct_chg[idx2]
                print('stock:%s ,earn:%f' %(stock_list[idx2],earn))
                earns+=earn
        
        return earns
        